In [1]:
import pandas as pd
from test_tools import evaluate_response


In [2]:
qa_df = pd.read_csv("chatbot_15_10.csv")
qa_df.head()


,Ignore,Question,Ideal Answer,Generated Answer,Retrieved Content,Alirio Score,Bam Score,Isaias Score,Dallan Score,Rick Score,Avg Score,Notes
0,NaN,How do I know if a student has a scholarship?,Missionaries can see if a student has a schola...,"To determine if a student has a scholarship, y...",node_id: 1\nhttps://sites.google.com/view/bloo...,5,5,5,5.0,5.0,5.0,NaN
1,NaN,How do I know if a student is registered for a...,There is not a way for Missionaries to verify ...,To determine if a student is registered for an...,node_id: 1\nhttps://missionaries.prod.byu-path...,1,1,1,1.0,1.0,1.0,NaN
2,NaN,How do I know if a student is a returned missi...,Missionaries can see if a student is a returne...,To determine if a student is a returned missio...,node_id: 1\nhttps://missionaries.prod.byu-path...,5,5,5,5.0,5.0,5.0,NaN
3,NaN,How do I know if student is member of the church?,Missionaries can see the Church membership sta...,To determine if a student is a member of the c...,node_id: 1\nhttps://missionaries.prod.byu-path...,5,5,5,5.0,4.0,4.8,NaN
4,NaN,What information should I track for each student?,The most important things Missionaries should ...,Tracking information for each student is cruci...,node_id: 1\nhttps://missionaries.prod.byu-path...,5,5,5,5.0,5.0,5.0,NaN


In [3]:
len(qa_df)


153

In [4]:
# drop rows with values in "Ignore"
qa_df = qa_df[qa_df["Ignore"].isnull()]
len(qa_df)


138

In [5]:
def get_ia_evaluation(row):
    question = row["Question"]
    ideal_answer = row["Ideal Answer"]
    generated = row["Generated Answer"]
    nodes = row["Retrieved Content"]

    ia_evaluation = evaluate_response(
        question=question,
        ideal=ideal_answer,
        generated=generated,
        nodes=nodes,
    )
    ia_score, ia_explanation = ia_evaluation.split(" - ")
    # get only the number inside ()
    ia_score = ia_score.split("(")[1].split(")")[0]

    return pd.Series([ia_explanation, ia_score], index=["ia_explanation", "ia_score"])


In [6]:
qa_df[["ia_explanation", "ia_score"]] = qa_df.apply(get_ia_evaluation, axis=1)
qa_df["ia_score"] = pd.to_numeric(qa_df["ia_score"])


In [7]:
qa_df.head()


,Ignore,Question,Ideal Answer,Generated Answer,Retrieved Content,Alirio Score,Bam Score,Isaias Score,Dallan Score,Rick Score,Avg Score,Notes,ia_explanation,ia_score
0,NaN,How do I know if a student has a scholarship?,Missionaries can see if a student has a schola...,"To determine if a student has a scholarship, y...",node_id: 1\nhttps://sites.google.com/view/bloo...,5,5,5,5.0,5.0,5.0,NaN,The generated answer provides the necessary in...,5
1,NaN,How do I know if a student is registered for a...,There is not a way for Missionaries to verify ...,To determine if a student is registered for an...,node_id: 1\nhttps://missionaries.prod.byu-path...,1,1,1,1.0,1.0,1.0,NaN,The generated answer provides steps that are n...,1
2,NaN,How do I know if a student is a returned missi...,Missionaries can see if a student is a returne...,To determine if a student is a returned missio...,node_id: 1\nhttps://missionaries.prod.byu-path...,5,5,5,5.0,5.0,5.0,NaN,The differences between the ideal answer and t...,5
3,NaN,How do I know if student is member of the church?,Missionaries can see the Church membership sta...,To determine if a student is a member of the c...,node_id: 1\nhttps://missionaries.prod.byu-path...,5,5,5,5.0,4.0,4.8,NaN,The generated answer includes all the necessar...,5
4,NaN,What information should I track for each student?,The most important things Missionaries should ...,Tracking information for each student is cruci...,node_id: 1\nhttps://missionaries.prod.byu-path...,5,5,5,5.0,5.0,5.0,NaN,The generated answer includes all the necessar...,5


In [8]:
qa_df.to_csv("chatbot_answer_15_10.csv")


In [9]:
# Calculating the correlation between our scores and LLM's
correlation = qa_df[["Avg Score", "ia_score"]].corr(method="kendall")

print("Correlation between our scores and LLM's:")
print(correlation)


Correlation between our scores and LLM's:
           Avg Score  ia_score
Avg Score   1.000000  0.535801
ia_score    0.535801  1.000000
